# FoldTree Jump、Cutpoint and AtomTree

上一节中，我们介绍了Rosetta中坐标的处理方式Rosetta的基本概念，及其顺序性。这一节中，我们将继续介绍：   
(1)FoldTree的另两个特性：跳跃性和可切割性。  
(2)基于Jump的Rigid-body Transformations

In [ ]:
# 初始化pyrosetta，使用pyrosetta必须要做的一件事情
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.teaching import *
from pyrosetta.toolbox import cleanATOM
# load
import nglview
from pyrosetta import init, pose_from_pdb
init()

## FoldTree的跳跃性和可切割性

上一节中我们已经提到了FoldTree的另外两个特性：跳跃性和可切割性，这一节我们将详细的介绍他们。

### FoldTree的跳跃性

在FoldTree的概念中，蛋白质的所有残基都是"相互连接"的，上一节我们已经提过，在FoldTree中当蛋白质有多条链组成时，尽管一条链中的残基都是共价连接，多条链之间实际上并不存在实际的共价连接。此时，FoldTree中表示这种虚拟共价连接的即为Jump。   
Jump实际上是一个针对蛋白的刚性的变化，可以视为为一种虚拟的共价连接。如同跳跃一般，可以建立一种远程的上下游关系。这种描述可能会有一些抽象，我们通过实例的方式来进行讲解。

首先我们使用了cleanATOM方法来对我们的目标PDB进行了清洗。并得到了该pose的FoldTree。

In [ ]:
#首先读取data文件中的PDB：1V74文件，并进行Clean
# 首先clean data文件夹中的PDB文件，并在该文件夹中生成对应的.clean.pdb
cleanATOM("./data/1v74.pdb")
#读取clean过的pdb文件
pwd = os.getcwd()
pose = pose_from_file(os.path.join(pwd,'./data/1V74.clean.pdb'))
pose_copy = pose.clone() #克隆一个pose，用于接下来FoldTree变化的示例
pose_copy_ii = pose.clone() #克隆一个pose，用于接下来FoldTree变化的示例
pose_copy_refold = pose.clone() # 克隆一个pose，用于接下来FoldTree的变化
# 输出该pose的FoldTree
print("PDB:1V74: ",pose.fold_tree())
print(f"The size of the foldtree (number of edges) is {pose.fold_tree().size()}") #Foldtree 的尺寸（edge的个数）
print(f"The number of the jump is {pose.fold_tree().num_jump()}") #Foldtree 的jump点的个数

为了进一步说明Jump的作用，我们通过Rosetta和PyMol的连用来进行可视化，便于讲述内容。  
前面已经讲过，Rosetta是可以和PyMol连用，来可视化蛋白Pose的。实际上，作为pose的一种属性，FolTree相关的氨基酸可以进行可视化，通过PyMOLMover的send_foldtree方法来完成。

**注意：PYMOLRosettaServer.py中有错误修正的脚本如下。但仍然没有复现其关于loop区显示的gong e**

<img src="./imgs/PyMOLRosettaServer_correct.png" width = "600" height = "300" align=center /> 

In [ ]:
# 可视化pose中的FoldTree在PyMol当中
from pyrosetta.rosetta.protocols.moves import PyMOLMover
pymover = PyMOLMover()
#将1V74的Pose发送至Pymol，并在Pymol中显示
# #将FoldTree发送至Pymol，并在Pymol中显示FoldTree
pymover.apply(pose) #在PyMol中显示fold_tree
pymover.send_foldtree(pose, pose.fold_tree()) #将fold_tree发送至pymol中
pymover.apply(pose) #在PyMol中显示fold_tree
# nglview.show_rosetta(pose) #使用nglview来显示pose

可以看到，图中灰色代表都是普通的edge中的氨基酸，而橙色的基团，则是指的是Jump点的氨基酸，红色则指的是cutpoint氨基酸。cutpoint会在下文中讲到

<img src="./imgs/fold_tree_one_example_1v74.png" width = "600" height = "300" align=center /> 

可以看到该Pose中含有3个Edge，一个Jump。与上一章介绍的一样，其中，从1-107、108-194最后一位为-1，因此这两个edge为共价连接，而1和108号残基，即该蛋白的A链和B链的N端由一个Jump连接，即一个虚拟的linker。Jump在被定义后，其前后会被定义为一个完整的FoldTree。此时A链的1号残基为B链108-194残基的上游

<img src="./imgs/first_jump.png" width = "600" height = "300" align=center /> 

In [ ]:
print(pose.phi(1)) #打印第一个残基的phi角的度数
pose.set_phi(1,70) #设置第一个残基的phi角为70度
pose.dump_pdb('./data/1v74_change_1_residue.pdb') #保存好改变角度的phi角
nglview.show_rosetta(pose) #使用nglview来查看pose

该pose的FoldTree的Jump种，只有1号残基和108-194号残基的即有上下游关系。当我们改变A链1号残基的phi角时，B链的构象也发生了变化，但改变A链其他的残基的几何参数时，不会引起B链构象的变化

In [ ]:
#将上面clone的构象用于修改第二个残基phi角的改变
print(pose_copy.phi(2)) #打印第二个残基phi角的度数
pose_copy.set_phi(2,70)  #设置第二个残基的phi角为70度
pymover.apply(pose_copy)
# pymover.send_foldtree(pose, pose.fold_tree())
nglview.show_rosetta(pose_copy)

当我们重新定义定义了Jump之后，因为Jump点前后视为一个完整FoldTree，这时，在FoldTree中所有的原子上下游关系进行重置。如下模块所示，此时我们重新定义了如下图所示该pose的FoldTree的Jump。   
此时我们设置了一个88-158的Jump点。可以看到，B链的上下游关系完全改变了，这也就体现了Jump点的上下游的性质：（1）Jump点的上游端点残基所在edge的按照图中的树的方向，很容易去理解。此时B链的158->108和158->194的上游为A链的1->88，而两者互相之间并无上下游的关系，与88->107这一段FoldTree也并无上下游关系

<img src="./imgs/second_jump_first_dir.png" width = "600" height = "300" align=center /> 

**！注意FoldTree中的设定一定要注意顺序性，FoldTree的树是有向的，即实际的上下游关系。如果不按照这种上下游关系进行建立就会报错**

In [ ]:
# 设置一个foldtree对象的实例，并将它读回pose中更新foldtree
ft = FoldTree() #设置一个空的foldtree对象
#按照如图所示的FoldTree的方向进行定义
ft.add_edge(1, 88, -1)
ft.add_edge(88, 107, -1)
ft.add_edge(88, 158, 1)
ft.add_edge(158, 108, -1)
ft.add_edge(158, 194, -1)
print(ft.check_fold_tree())
pose_copy_refold.fold_tree(ft)
pymover.apply(pose_copy_refold) #在PyMol中显示fold_tree
pymover.send_foldtree(pose_copy_refold, pose_copy_refold.fold_tree()) #将fold_tree发送至pymol中
pymover.apply(pose_copy_refold) #在PyMol中显示fold_tree
pose_copy_refold.dump_pdb('1v74_change_0_residue_cutpoint_refold.pdb') 

<img src="./imgs/fold_tree_one_example_1v74_refold.png" width = "600" height = "300" align=center /> 

可以看到，图中的Jump的位置发生了变化

这时候我们分别使用两种变化，即改变2号残基和89号残基的变化。来看pose的改变，

In [ ]:
#修改2号残基后pose的变化，B链的构象的变化
pose_copy_refold_i = pose_copy_refold.clone()
print(pose_copy_refold_i.phi(2))
#保存好改变角度的phi角
pose_copy_refold_i.set_phi(2,90)
pose_copy_refold_i.dump_pdb('./data/1v74_change_1_residue_cutpoint_refold.pdb') 
nglview.show_rosetta(pose_copy_refold_i)

<img src="./imgs/fold_tree_foldtreeii_1_1v74.png" width = "600" height = "300" align=center /> 

黄色的pose为原本的pdb，蓝色为改变phi角的pose。可以看到，两条链的构象都发生了变化

In [ ]:
#修改89号残基后pose的变化，并无引起B链的构象的变化
pose_copy_refold_ii = pose_copy_refold.clone() #从最初的pose克隆一个新的pose
print(pose_copy_refold_ii.phi(89)) #打印第89个残基的phi角
pose_copy_refold_ii.set_phi(89,70) #改变第89个残基的phi角为70度
pose_copy_refold_ii.dump_pdb('./data/1v74_change_2_residue_cutpoint_refold.pdb')  #保存为一个新的pdb文件
nglview.show_rosetta(pose_copy_refold_ii) #利用nglview显示这个pose的构象

<img src="./imgs/fold_tree_foldtreeii_2_1v74.png" width = "600" height = "300" align=center /> 

与上面的表示相同，这一个89之后的残基与B链无上下游关系，因此只有A链上的89号之后的残基发生了构象变化，其余的都没有变化。

**!注意：不要因为上面的示例，就以为Jump只能表示两段序列之间一种虚拟的上下游关系**   
Jump的设定也可以出现在一条肽链中，如下例所示，在设定一个8肽序列，我们建立一个3-6的Jump

In [ ]:
#由sequence建立一个pose
pose_example = pose_from_sequence('KPALNALN')
# 设置一个foldtree对象的实例，并将它读回pose中更新foldtree
ft = FoldTree() #设置一个空的foldtree对象
ft.add_edge(1, 3, -1) #增加一个5号残基到1号残基的共价连接的edge
ft.add_edge(3, 6, 1)
ft.add_edge(3, 6, -1)
ft.add_edge(6, 8, -1)
pose_example.fold_tree(ft) #更新pose的fold_tree
print(pose_example.fold_tree()) #输出foldtree
#如图所示，3-6以及6一8的上游都是1-3残基
pymover.apply(pose_example) #在PyMol中显示fold_tree
pymover.send_foldtree(pose_example, pose_example.fold_tree()) #将fold_tree发送至pymol中
pymover.apply(pose_example) #在PyMol中显示fold_tree

<img src="./imgs/third_foldtree.png" width = "600" height = "300" align=center /> 

<img src="./imgs/fold_tree_example_1_peptide.png" width = "600" height = "300" align=center /> 

在一个多肽里进行使用的时候，同样也要注意FoldTree的方向性

In [ ]:
# 设置一个foldtree对象的实例，并将它读回pose中更新foldtree
ft = FoldTree() #设置一个空的foldtree对象
#增加一个1号残基到3号残基的共价连接的edge
ft.add_edge(1, 3, -1)
#增加一个3号残基到6号残基的共价连接的edge
ft.add_edge(3, 6, -1)
#增加一个3号残基到6号残基的一个Jump
ft.add_edge(6, 3, 1)
#增加一个6号残基到8号残基的共价连接的edge
ft.add_edge(6, 8, -1)
pose_example.fold_tree(ft) #更新pose的fold_tree
print(pose_example.fold_tree()) #输出foldtree

<img src="./imgs/simple_foldtree.png" width = "600" height = "300" align=center /> 

### FoldTree的可切割性

FoldTree是可以在edge中进行对点的切割，即设置cutpoint，在快速设置loop时十分有用。   
总体来说，Cutpoint就是FoldTree中的不连续点，相对于peptide edge的连续性来说，cutpoint的出现相当于一把剪刀，将非peptide edge内氨基酸的连接关系切断，也就是FoldTree内EDGE的上下游关系在此处被切断。

我们可以使用对应pose的FoldTree的num_cutpoint属性来看cutpoint的数量，并通过cutpoint属性来看对应cutpoint的位置，以及is_cutpoint判断是否为cutpoint

In [ ]:
#看对应pose中cutpoint的属性
pose = pose_from_file('./data/1V74.clean.pdb')
print(f"The FoldTree of the 1V74: {pose.fold_tree()}")
print(f"The number of the cutpoint in 1V74 is : {pose.fold_tree().num_cutpoint()}")#说明1V74的cutpoint只有一个
print(f"The cutpoint in 1V74 is : {pose.fold_tree().cutpoint(1)}")#说明PDB：1V74的断点再107号残基处

还是以这张图为例，可以看到，再FoldTree中107和108的连接关系并没有出现在FoldTree的任何一个EDGE当中，因此其被相当于一个cutpoint

<img src="./imgs/first_jump.png" width = "600" height = "300" align=center /> 

cutpoint实际上是对于pose中residue的操作，给予化学键的变化，针对设置断点的上下游，给予残基一个patch来实现的。具体的方式可以如下:   
[1]chemical中的add_variant_type_to_pose_residue，同时通过VariantType定义的断点上下游来进行来完成。这里我们还是以这个八肽来做示范，尽管此时FoldTree中并没有cutpoint的发现，但实际上在patch上已经有了cutpoint的定义，如下所示。

add_variant_type_to_pose_residue方法的变量分别是：   
【1】pose：一个处理好的pose；   
【2】VariantType：定义好的化学属性，在这里使用的是CUTPOINT_LOWER或CUTPOINT_UPPER，以此定义cutpoint的上下游   
【3】cut_res：被切割的残基的编号。

**!TODO：发现并没有起到cutpoint的作用，不知道为什么**

In [ ]:
# add cutpoint
from pyrosetta.rosetta.core.pose import add_variant_type_to_pose_residue
from pyrosetta.rosetta.core.chemical import VariantType
# 输出该pose的FoldTree
pose_example = pose_from_sequence('KPALNALN')
print(f"The number of cutpoint in the PDB:1V74:{pose_example.fold_tree().num_cutpoint()}")
nglview.show_rosetta(pose_example)
#在5、6号残基上给予一个cutpoint，分别定义5号残基为lower（上游）、6号残基为upper（下游）.
cut_res = 5
add_variant_type_to_pose_residue(pose_example, VariantType.CUTPOINT_LOWER, cut_res) #给指定残基加上上游的patch
add_variant_type_to_pose_residue(pose_example, VariantType.CUTPOINT_UPPER, cut_res+1) #给指定残基加上下游的patch
#此时FoldTree中并未有cutpoint的显示。
print(f"The number of cutpoint in the PDB:1V74:{pose_example.fold_tree().num_cutpoint()}") #输出其固定
print(pose_example.residue(1).annotated_name())
print(pose_example.residue(1).annotated_name())

此时，FoldTree的cutpoint的并没有显示，（推测）**因为目前的变化是在Rosetta的residue的层级上进行的操作，针对residue增加patch，而非直接对FoldTree进行操作，因此不会在这里显示**

一件有趣的事情是，cutpoint在定义时实际上也可以有一个方向性。再定义上下的cutpoint时，6-7和7-6的表示是不同的。   
**注：这个方向性不知道有什么样的影响。**

In [ ]:
# 建立该pose的FoldTree，并输出该pose的FoldTree和cut point
pose_example = pose_from_sequence('KPALNALN') #从sequence直接得到肽链的pose
# 设置一个foldtree对象的实例，并将它读回pose中更新foldtree
ft = FoldTree() #设置一个空的foldtree对象
#增加一个1号残基到3号残基的共价连接的edge
ft.add_edge(1, 3, -1)
#增加一个3号残基到6号残基的共价连接的edge
ft.add_edge(3, 6, -1)
#增加一个3号残基到6号残基的一个Jump
ft.add_edge(3, 6, 1)
#增加一个6号残基到8号残基的共价连接的edge
ft.add_edge(6, 8, -1)
pose_example.fold_tree(ft) #更新pose的fold_tree
print(f"The number of cutpoint in the PDB:1V74:{pose_example.fold_tree().num_cutpoint()}") # 输出该pose的FoldTree
print(pose_example.fold_tree()) #输出foldtree
pose_example_ii = pose_example.clone()
pose_example_iii = pose_example.clone()
pose_example_iv = pose_example.clone()
pose_example.dump_pdb('./data/peptide_0_cutpoint_example.pdb')

In [ ]:
#在7、8号残基上给予一个cutpoint，分别定义7号位lower、8号为upper.
cut_res = 6
add_variant_type_to_pose_residue(pose_example_ii, VariantType.CUTPOINT_UPPER, cut_res)
add_variant_type_to_pose_residue(pose_example_ii, VariantType.CUTPOINT_LOWER, cut_res+1)
#此时FoldTree中并未有cutpoint的显示。
print(pose_example_ii.residue(6).annotated_name())
print(pose_example_ii.residue(7).annotated_name())
pose_example_ii.set_phi(3,90)
pose_example_ii.dump_pdb('./data/peptide_1_cutpoint_example.pdb')

**不知道为什么：在已有的FoldTree中加入cutpoint的时候，在N端末的氨基酸不能作为cutpoint的下游，而C端的末的氨基酸不能作为cutpoint的下游。**

In [ ]:
#在6、7号残基上给予一个cutpoint，分别定义7号位lower、8号为upper.
cut_res = 7
add_variant_type_to_pose_residue(pose_example_iii, VariantType.CUTPOINT_LOWER, cut_res+1)
add_variant_type_to_pose_residue(pose_example_iii, VariantType.CUTPOINT_UPPER, cut_res)
#此时FoldTree中并未有cutpoint的显示。
print(f"The number of cutpoint in the PDB:1V74:{pose_example_iii.fold_tree().num_cutpoint()}")
print(pose_example_iii.residue(cut_res).annotated_name())
print(pose_example_iii.residue(cut_res+1).annotated_name())

In [ ]:
#在6、7号残基上给予一个cutpoint，分别定义7号位lower、8号为upper.
cut_res = 1
add_variant_type_to_pose_residue(pose_example_iv, VariantType.CUTPOINT_LOWER, cut_res+1)
add_variant_type_to_pose_residue(pose_example_iv, VariantType.CUTPOINT_UPPER, cut_res)
#此时FoldTree中并未有cutpoint的显示。
print(f"The number of cutpoint in the PDB:1V74:{pose_example_iv.fold_tree().num_cutpoint()}")
print(pose_example_iv.residue(cut_res).annotated_name())
print(pose_example_iv.residue(cut_res+1).annotated_name())

[2]FoldTree对象中的new_jump方法。再加入一个新的jump的同时，再给予一个新的断点。这样的操作可以更加方便的加入Jump中的新的断点

In [ ]:
# 输出该pose的FoldTree
pose_example = pose_from_sequence('KPALNALN')
print(f"The number of cutpoint in the PDB:1V74:{pose_example.fold_tree().num_cutpoint()}")
# 设置一个foldtree对象的实例，并将它读回pose中更新foldtree
ft = FoldTree() #设置一个空的foldtree对象
#增加一个1号残基到3号残基的共价连接的edge
ft.add_edge(1, 3, -1)
#增加一个3号残基到6号残基的共价连接的edge
ft.add_edge(3, 6, -1)
#增加一个3号残基到6号残基的一个Jump
ft.new_jump(3, 6, 5) #分别代表着Jump的pos1和pos2以及自定义的cutpoint
#增加一个6号残基到8号残基的共价连接的edge
ft.add_edge(6, 8, -1)
pose_example.fold_tree(ft) #更新pose的fold_tree
print(pose_example.fold_tree()) #输出foldtree
#此时FoldTree中并未有cutpoint的显示。
print(f"The number of cutpoint in the PDB:1V74:{pose_example.fold_tree().num_cutpoint()}")#得到该FoldTree中的断点数
print("The residue for cutpoint is", pose_example.fold_tree().cutpoint(1))
print(pose_example.residue(5).annotated_name())#此时并没有patch的产生
nglview.show_rosetta(pose_example)
pymover.apply(pose_example) #在PyMol中显示fold_tree
pymover.send_foldtree(pose_example, pose_example.fold_tree()) #将fold_tree发送至pymol中
pymover.apply(pose_example) #在PyMol中显示fold_tree
pose_example.dump_pdb('./data/peptide_1_residue_cutpoint.pdb')  #保存为一个新的pdb文件

如图，中间多了一个cutpoint

<img src="./imgs/fold_tree_example_2_peptide.png" width = "600" height = "300" align=center /> 

In [ ]:
pose_example.set_phi(3,90)
pose_example.dump_pdb('./data/peptide_2_residue_cutpoint.pdb')  #保存为一个新的pdb文件

<img src="./imgs/fold_tree_cutpoint_2_peptide.png" width = "600" height = "300" align=center /> 

黄色为之前的肽链，蓝色为设置cutpoint之后的肽链，很显然，在cutpoint之后，并没有发现刚性的构象变化。

cutpoint具有如下**两个特性**：   

1.没有peptide edge相连的氨基酸为cutpoint；  

In [ ]:
ft = FoldTree() #设置一个空的foldtree对象
#增加一个1号残基到3号残基的共价连接的edge
# 设置一个foldtree对象的实例，并将它读回pose中更新foldtree
ft = FoldTree() #设置一个空的foldtree对象
#增加一个1号残基到3号残基的共价连接的edge
ft.add_edge(1, 3, -1)
#增加一个3号残基到6号残基的共价连接的edge
ft.add_edge(3, 6, -1)
#增加一个3号残基到6号残基的一个Jump
ft.add_edge(3, 6, 1)
#增加一个6号残基到8号残基的共价连接的edge
ft.add_edge(6, 8, 2)
ft.add_edge(8, 7, -1)
pose_example.fold_tree(ft) #更新pose的fold_tree
print(pose_example.fold_tree().num_jump()) #查看FoldTree中Jump的数目
print(pose_example.fold_tree().num_cutpoint()) #查看FoldTree中的cutpoint
print(pose_example.fold_tree().cutpoint(1)) #查看cutpoint的位置
print(pose_example.residue(6).annotated_name()) #在FoldTree中的cutpoint并没有加上annotated_name

2.无论在上游还是下游区间中，氨基酸序号高者均为cutpoint；(但计算foldtree中cutpoints的总数时却不算)

**！TODO:不知道这些数字什么意思。。。**

In [ ]:
print(pose_example.fold_tree().cutpoint(1))
print(pose_example.fold_tree().cutpoint(2))
print(pose_example.fold_tree().cutpoint(3))
print(pose_example.fold_tree().cutpoint(4))
print(pose_example.fold_tree().cutpoint(5))

## Rigid-body transformations

###  引言

<img src="./imgs/rigid-body.png" width = "600" height = "300" align=center /> 

蛋白质-蛋白质对接是从其单体成分开始的复杂结构的预测，搜索空间可能非常大。Rosetta中有大量的mover来应对docking中的几何参数的变化。   
最基础的docking move是刚体的变换，包括**平移**和**旋转**，每一个刚体move都需要知道，哪一部分进行移动，以及哪一部分进行固定。而这一部分的move，其实主要是靠FoldTree中的Jump方法来建立，并最终完成各种move。下面根据实例来进行示范并讲解。

### 及已经写入protocol的FoldTree建立方法

In [ ]:
# 首先，设置两个pdb的clone用于接下来的处理，查看处理的变化
pose = pose_from_file('./data/1V74.clean.pdb') #从pdb文件中读取pose
print(f'The FoldTree of PDB:1V74:{pose.fold_tree()}') #获取该pose的FoldTree
starting_pose = pose.clone() #使用pose的clone方法，复制一个用做接下来处理的pose
cen_pose = pose.clone() #使用pose的clone方法，复制一个用于一个原子粒度move的处理的pose

<img src="./imgs/first_jump.png" width = "600" height = "300" align=center /> 

此时，该蛋白的A链和B链是由1-108这样的Jump来进行连接的。上面我们已经讲过Jump，而Jump实际上是由rotation和translation组成的自由度变化。也就是说：   
此时108-194会根据这个Jump进行对应1号残基的rotation和translation的变化。

In [ ]:
print("The rotation matrix is:\n", pose.jump(1).get_rotation())
print("The translation vector is:\n", pose.jump(1).get_translation())

但我们并不希望是这样的刚性自由度处理方式，而是由自己定义的方式来进行。如下图所示，我们想跟据链的质心来定义FoldTree，88和158分别是两条链的质心，我们想以如下的FoldTree的方式来进行刚性变化。

<img src="./imgs/second-jump.png" width = "600" height = "300" align=center /> 

上文中，实际上我们已经通过新建FoldTree的方式实现了这种变化。但实际上，因为在docking经常带有这样的操作，在pyrosetta的protocol中已经有打包好的protocol提供出来建立这样的FoldTree，在这里就是setup_foldtree方法

setup_foldtree主要处理的是刚性变化时的一个蛋白不同链之间的Jump关系的建立。其输入变量为：   
【1】pose：一个蛋白复合体的pose；    
【2】partner_chainID：根据pdb_chain的编号，定义好的链与链之间是否是刚性的。例如这里"A_B"是指A_B之间存在Jump，而如果有一个具有三条链的蛋白，再建立foldtree的时候，使用“AB_C”，则表示A、B链共同是刚性的，而C链会和A、B链建立Jump关系，两者之间会发现rotation或translation。   
【3】movable_jumps：定义jump的编号

这种方法默认会对需要操作的链与链的质心进行来定义jump

我们使用上述方法，发现其FoldTree发现了变化

In [ ]:
from pyrosetta.rosetta.protocols.docking import setup_foldtree
setup_foldtree(starting_pose, "A_B", Vector1([1]))
print(pose.fold_tree()) #原pose的FoldTree
print(starting_pose.fold_tree()) #经过修改后的FoldTree

此时，第一个jump处的rotation matrix以及translation vector，发生了变化。

In [ ]:
print("The rotation matrix is:\n", starting_pose.jump(1).get_rotation())
print("The translation vector is:\n", starting_pose.jump(1).get_translation())

当然，rotation需要一个旋转轴，tranlation则是需要平移向量，两者是如何根据Jump进行定义的呢？

### (选修)Rosetta中的笛卡尔坐标处理

在上一讲中，我们实际上已经讲解了内坐标以及基于内坐标处理的表示动力学的数据结构FoldTree。但在计算多个蛋白的移动的时候，我们使用Jump进行虚拟连接时，因为本身并不是一个实际的上下游的处理关系，我们实际上需要将内坐标转换为3D坐标，来得到对应的rotation以及translation。接下来就是Rosetta对这种情况的处理，也就是其为了联系内坐标与笛卡尔坐标的数据结构AtomTree

#### AtomTree简介

AtomTree是Rosetta内置的一种数据结构，其主要作用就是用于pose的内坐标和笛卡尔坐标的对齐。

AtomTree 提供了一个更通用的框架来模拟生物大分子系统。顾名思义，与 FoldTree 中基于残基的表示相比，它以树状拓扑表示系统中的所有原子。原子通过键距、键角和扭转角度以及刚体变换相互连接，所有这些都是系统的潜在自由度。（FoldTree 可以被认为是一种特殊的 AtomTree，其中键长/角度是固定的，并且只允许扭转/刚体 DOF）。AtomTree 不仅允许用户处理更多的自由度，其基于原子的表示在处理具有任意原子连接的非聚合物型分子（例如小化学配体）方面也具有很大优势。这里我们只是借用AtomTree来讲解Jump的解释，而不会介绍基于AtomTree来对蛋白的自由度进行编辑的操作。

在Rosetta中，对于固定pose的AtomTree，是以atom_tree()来调用的。

In [ ]:
# 输出该pose的FoldTree
pose_example = pose_from_sequence('KPALNALN')
print(f"The number of cutpoint in the PDB:1V74:{pose_example.fold_tree().num_cutpoint()}")
# 设置一个foldtree对象的实例，并将它读回pose中更新foldtree
ft = FoldTree() #设置一个空的foldtree对象
#增加一个1号残基到3号残基的共价连接的edge
ft.add_edge(1, 3, -1)
#增加一个3号残基到6号残基的共价连接的edge
ft.add_edge(3, 6, -1)
#增加一个3号残基到6号残基的一个Jump
ft.add_edge(3, 6, 1)
# ft.add_edge(6, 3, 1)
# ft.new_jump(3, 6, 5) #分别代表着Jump的pos1和pos2以及自定义的cutpoint
#增加一个6号残基到8号残基的共价连接的edge
ft.add_edge(6, 8, -1)
pose_example.fold_tree(ft) #更新pose的fold_tree
example_atom_tree = pose_example.atom_tree() #得到这个时候的AtomTree

与一般的树结构类似，AtomTree同样需要一个root原子在树的底端。

In [ ]:
# 查看AtomTree的root原子
roots = example_atom_tree.root() #获取root原子
roots_id = roots.id() #获取root原子的AtomID对象
print(roots_id)
root_residue_id = roots_id.rsd() #获取root原子所在的残基的编号
root_atom = roots_id.atomno() #获取root原子在所在残基中的原子变好
atom_root_name = pose_example.residue(root_residue_id).atom_name(root_atom) #获取该root原子的名称
print(f'The root atom is the "{atom_root_name}" atom in the {root_residue_id}th residue ') 

可以看到，这个pose的root原子为其1号残基的root原子。我们可以通过定义该原子的AtomID的方式来得到该原子。

In [ ]:
from rosetta.core.id import AtomID
atomid = AtomID(root_atom, root_residue_id)
atom = example_atom_tree.atom(atomid)

既然有了root原子，那必定有其子节点

In [ ]:
# 查看AtomTree的root原子
for i in range(roots.n_children()): # n_children表示该root原子子节点的个数
    child_node = roots.child(i)
    residue_id = child_node.id().rsd()
    atom_id = child_node.id().atomno()
    print(residue_id, atom_id)
    atom_child_name = pose_example.residue(residue_id).atom_name(atom_id)
    print(f'The {i+1}th child atom is the "{atom_child_name}" atom in the {residue_id}th residue ') 

In [ ]:
atomid = AtomID(1, 1)
atom_1 = example_atom_tree.atom(atomid)
print(atom_1.n_children())
for i in range(atom_1.n_children()):
    child_node_1 = atom_1.child(i)
    print(child_node_1.id().rsd(), child_node_1.id().atomno())
    print(pose_example.residue(child_node_1.id().rsd()).atom_name(child_node_1.id().atomno()))

In [ ]:
atomid = AtomID(2, 3)
atom_1 = example_atom_tree.atom(atomid)
print(atom_1.n_children())
for i in range(atom_1.n_children()):
    child_node_1 = atom_1.child(i)
    print(child_node_1.id().rsd(), child_node_1.id().atomno())
    print(pose_example.residue(child_node_1.id().rsd()).atom_name(child_node_1.id().atomno()))

此时，我们就得到了AtomTree的第一层的子节点。利用递归式，就可以得到整个AtomTree的结构，这里就不再展示了。

#### Stub

而后，Rosetta将几何结构转化为局部坐标，称为Stub。每一个Stub包括stub atom在Rosetta默认坐标系中的笛卡尔坐标，以及将坐标从局部坐标系旋转到全局坐标系的维度为3x3的旋转矩阵，数学表示的意思是：
实际上一个Stub表示的是：   
原点点V的Rosetta坐标系中的坐标以及正交坐标系转换矩阵M。其建立方式为由四个点定义，一个是中心，另外三个，即另外两个原子及两个向量叉乘的点。用于计算正交框架。

In [ ]:
from pyrosetta.rosetta.core.id import StubID, AtomID
# 构建以root为原点的骨架三元组N-CA-C的Stub
stub_id = StubID(AtomID(2, 1), AtomID(1, 1), AtomID(3, 1))
stub = example_atom_tree.stub_from_id(stub_id)
center = stub.center()
matrix = stub.M
print("The stub's Center:\n", center)
print("The Matrix of the Stub:\n", matrix)
print()

以目前的Stub为例。首先这三个院子的坐标为：

In [ ]:
print(pose_example.residue(1).atom(2))
print(pose_example.residue(1).atom(1))
print(pose_example.residue(1).atom(3))

可以看到，center实际上就是上面的root原子，也就是我们在定义定义stub的时候，输入的第一个原子

In [ ]:
# 建立三个点坐标
import numpy as np
dot_vector_1 = np.array([1.458, 0, 0])
dot_vector_2 = np.array([0, 0, 0])
dot_vector_3 = np.array([2.00885, 1.42017, 0])

In [ ]:
# 得到两个向量，以此作为新的局部坐标的X轴、Y轴
vector_1 = (dot_vector_2 - dot_vector_1).reshape(1,-1)
vector_2 = (dot_vector_3 - dot_vector_1).reshape(1,-1)
# 利用矩阵叉乘求第三个基
vector_3 = np.cross(vector_1,vector_2).reshape(1,-1)

In [ ]:
# 求这个坐标系的基矩阵
basis_matrix = np.concatenate([vector_1, vector_2, vector_3], axis=0)

In [ ]:
print(basis_matrix)

In [ ]:
rotation_matrix = np.array([[0.9999999999999999,0.000000000000000,0.000000000000000],
                            [0.000000000000000,0.9999999999999712,2.393520176095881E-07],
                            [0.000000000000000, -2.393520176095880E-07,0.9999999999999714]])
print('The corrected coordinates is:\n',basis_matrix*rotation_matrix)

可以看到，旋转矩阵的作用是将原本的X轴、Y轴和Z轴通过旋转和投影，变成一个保留对应轴上投影的一个正交基的操作。

####  Jump

上面提到了，Stub实际上是将全局坐标转换为局部坐标的方式。Jump与Stub类似，但实际上，他是定义了Stub之间的关系，即一个Stub到另外一个Stub的旋转和平移。因此，Jump指定了一组相对于另一组的六自由度：六自由度，刚体位置和方向

在上文我们提到了root原子。实际上AtomTree的root原子和每个新的独立组的伪root原子，在Rosetta中都是 JumpAtom。这些原子都通过Jump与其与在树中的父节点相关，如果父节点是root节点，也包括和root节点的相关。   

在上面的例子中，jump是3->6，自然的，下一个独立组的起始残基就是6号残基，其伪root原子便是其CA原子，其也是一个JumpAtom。在这里我们其实可以看到，所有的Jump原子都是所设置的root氨基酸的CA原子。在这里除了root原子和6号残基的CA原子，其他都是BondedAtom

In [ ]:
atomid_origin = AtomID(2, 1)
atomid_up = AtomID(2, 6)
atomid_down = AtomID(2, 3)
print(f"The 1th residue's 2th atom is {example_atom_tree.atom(atomid_origin)}")
print(f"The 6th residue's 2th atom is {example_atom_tree.atom(atomid_up)}")
print(f"The 6th residue's 3th atom is {example_atom_tree.atom(atomid_down)}")
print(example_atom_tree.atom(atomid_up).get_stub().M)
print(example_atom_tree.atom(atomid_up).get_stub().v,'\n')
print(example_atom_tree.atom(atomid_down).get_stub().M)
print(example_atom_tree.atom(atomid_down).get_stub().v,'\n')

考虑两个局部坐标Stub之间的转换，从B的Stub出发（A 的输入存根）生成 A 的Stub，后乘B的Stub的M矩阵 M_B，矩阵以phi围绕X旋转，以theta角围绕 Z轴旋转，然后通过该矩阵平移B的存根中心v_B乘以向量 (d,0,0)。   

M_A = M_B * M_phi * M_theta   
v_A = v_B + M_A * (d,0,0)

In [ ]:
print(pose_example.jump(1).get_rotation())
print(pose_example.jump(1).get_translation())

In [ ]:
matrix = example_atom_tree.atom(atomid_down).get_stub().M * pose_example.jump(1).get_rotation()

In [ ]:
print(matrix)

可以看到Jump的rotation矩阵，就是M_phi * M_theta的结果矩阵。

**!TODO:Translation真的复现不出来**